In [1]:
from datasets import load_from_disk
from jinja2.lexer import ignored_tokens

dataset = load_from_disk("preprocessed_dataset/paired_tokenized_dataset")

for i in dataset["train"]:
    print(i)
    break

{'source_tokens': [38, 82, 79, 77, 221, 87, 72, 79, 77, 221, 69, 86, 69, 82, 89, 221, 70, 65, 77, 73, 76, 89, 221, 73, 78, 221, 72, 69, 65, 86, 69, 78, 221, 65, 78, 68, 221, 79, 78, 221, 69, 65, 82, 84, 72, 221, 84, 65, 75, 69, 83, 221, 73, 84, 83, 221, 78, 65, 77, 69, 14, 0, 0, 0], 'target_tokens': [90, 221, 75, 84, 79, 82, 128, 103, 72, 79, 221, 77, 128, 95, 221, 75, 65, 130, 123, 68, 128, 122, 221, 82, 79, 68, 221, 78, 65, 221, 78, 69, 66, 69, 83, 73, 65, 67, 72, 221, 73, 221, 78, 65, 221, 90, 69, 77, 73, 221, 83, 86, 79, 74, 69, 221, 77, 69, 78, 79, 12, 0, 0, 0]}


In [42]:
transformer = Transformer(config)
print(transformer)
print("Number of parameters:", sum(p.numel() for p in transformer.parameters()))

for i in dataset["train"]:
    batch = torch.tensor([i["source_tokens"]]).to(next(transformer.parameters()).device)
    target_tokens =torch.tensor([[10,20,10,20,10,20,10,20]]).to(next(transformer.parameters()).device)
    print(transformer(batch, target_tokens))
    print(transformer(batch, target_tokens))
    break

Transformer(
  (encoder): Encoder(
    (embedding): Embedding(
      (token_embedding): Embedding(10000, 32)
    )
    (blocks): ModuleList(
      (0): TransformerBlock(
        (multi_head_attention): MultiHeadAttention(
          (q_proj): Linear(in_features=32, out_features=32, bias=True)
          (k_proj): Linear(in_features=32, out_features=32, bias=True)
          (v_proj): Linear(in_features=32, out_features=32, bias=True)
          (out_proj): Linear(in_features=32, out_features=32, bias=True)
          (softmax): Softmax(dim=-1)
        )
        (feed_forward): Sequential(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): ReLU()
          (2): Linear(in_features=32, out_features=32, bias=True)
        )
        (layer_norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (layer_norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    

In [43]:
from tqdm import tqdm
import random

class Trainer:
    def __init__(self, model, dataset, learning_rate, batch_size):
        self.model = model
        
        self.dataset = dataset
        self.data_loader = torch.utils.data.DataLoader(dataset["train"], batch_size=batch_size)
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.loss = torch.nn.CrossEntropyLoss()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(self.device)
        
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)
    
    def load_model(self, path):
        self.model.load_state_dict(torch.load(path))
        
    def train(self, n_epochs):
        for epoch in range(n_epochs):
            self.train_epoch()
            self.save_model(f"checkpoint/epoch_{epoch}.pt")
                
    def train_epoch(self):
        epoch_loss = 0
        steps = 0
        for batch in (progress_bar := tqdm(self.data_loader)):
            source_tokens = torch.stack(batch["source_tokens"], dim=1).to(self.device)
            target_tokens = torch.stack(batch["target_tokens"], dim=1).to(self.device)
            #target_tokens = torch.stack([torch.tensor([10,20,10,20,10,20,10,20]) for i in target_tokens]).to(self.device)
            #if random.random() < 0.5:
            #    target_tokens = torch.tensor([[10, 10, 10] for i in target_tokens]).to(self.device)
            #else:
            #    target_tokens = torch.tensor([[20, 10, 20] for i in target_tokens]).to(self.device)
            
            predictions = self.model(source_tokens, target_tokens)[:, :-1]  # there is no ground truth for the last token
            print(predictions)
            loss = self.loss(predictions.reshape(-1, config["vocab_size"]), target_tokens.view(-1))
            print(loss)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            break
            
            epoch_loss += loss.item()
            if steps % 1000 == 0:
                print(f"target: {target_tokens[0]}")
                print(f"prediction: {torch.argmax(predictions[0], dim=-1)}")
            
            steps += 1
            progress_bar.desc = f"avg loss: {epoch_loss / steps:.4f}"
        
                
 
trainer = Trainer(transformer, dataset, 1e-6, 128)
trainer.train(1)
#trainer.load_model("checkpoint/epoch_0.pt")

  0%|          | 0/7035 [00:00<?, ?it/s]

tensor([[[5.3051e-05, 6.8222e-05, 1.2062e-04,  ..., 1.8118e-04,
          6.9246e-05, 8.5104e-05],
         [3.2716e-05, 5.3148e-05, 4.4545e-05,  ..., 1.2875e-04,
          7.7368e-05, 3.4301e-05],
         [8.5483e-05, 6.5093e-05, 6.6989e-05,  ..., 1.3055e-04,
          1.1358e-04, 2.0884e-05],
         ...,
         [3.3404e-05, 6.8917e-05, 9.3391e-05,  ..., 3.0210e-04,
          1.5110e-04, 6.3270e-05],
         [3.2458e-05, 4.5867e-05, 8.7771e-05,  ..., 2.1431e-04,
          3.3203e-04, 4.7935e-05],
         [4.5846e-05, 6.7635e-05, 1.3697e-04,  ..., 1.5476e-04,
          2.7897e-04, 6.2755e-05]],

        [[8.4789e-05, 9.3145e-05, 1.4359e-04,  ..., 1.6626e-04,
          1.2231e-04, 5.1772e-05],
         [2.8598e-05, 4.9335e-05, 3.5168e-05,  ..., 1.7987e-04,
          1.4791e-04, 2.4233e-05],
         [3.4207e-05, 8.9369e-05, 3.8548e-05,  ..., 2.8438e-04,
          2.4675e-04, 1.6461e-05],
         ...,
         [5.0140e-05, 5.9609e-05, 8.5691e-05,  ..., 1.7527e-04,
          2.683

In [7]:
from transformers import AutoTokenizer

class Translator:
    def __init__(self, model, source_tokenizer, target_tokenizer, search_strategy="greedy"):
        self.model = model
        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer
        
        if search_strategy == "greedy":
            self.search_strategy = self.greedy_search
        
    def translate(self, text, max_length=None):
        source_tokens = self.source_tokenizer(text, truncation=True, padding="max_length", max_length=64).input_ids
        source_tokens = torch.tensor(source_tokens).unsqueeze(0).to(next(self.model.parameters()).device)
        
        if max_length is None:
            max_length = self.model.config["max_seq_len"]
        
        encoder_output = self.model.encoder(source_tokens)
        
        translated_tokens = torch.empty(source_tokens.size()[0], 0, dtype=torch.long).to(source_tokens.device)
        for i in range(max_length):
            predictions = self.model.decoder(translated_tokens, encoder_output)
            translated_tokens = self.search_strategy(translated_tokens, predictions)
            if torch.all(translated_tokens[:, -1] == self.target_tokenizer.eos_token_id):
                break
        
        return self.target_tokenizer.batch_decode(translated_tokens)
        
    def greedy_search(self, translated_tokens, predictions):
        return torch.cat([translated_tokens, torch.argmax(predictions[:, -1, :], dim=-1).unsqueeze(0)], dim=-1)


source_tokenizer = AutoTokenizer.from_pretrained("preprocessed_dataset/source_tokenizer")
target_tokenizer = AutoTokenizer.from_pretrained("preprocessed_dataset/target_tokenizer")
source_tokenizer.pad_token = "<pad>" 
translator = Translator(transformer, source_tokenizer, target_tokenizer) 

translator.translate("hello")

['<|endoftext|>']

In [131]:
print(transformer.decoder.embedding.token_embedding.weight.data[10])
print(transformer.decoder.embedding.token_embedding.weight.data[20])

print(transformer.decoder.embedding.positional_embedding[0, 0])
print(transformer.decoder.embedding.positional_embedding[0, 2])


tensor([-0.9029, -0.2277,  0.8121,  0.4913, -0.1294,  2.2102,  1.9592,  0.1228,
        -0.1017, -0.0548,  0.7717, -0.7806, -1.6557,  0.2264, -0.1933,  0.7390,
         0.2000, -0.1708, -2.0923,  0.1391, -0.3606,  0.5193,  1.0636,  0.1898,
        -0.5848, -1.8388, -1.0696,  1.1762, -1.6640, -1.3206,  0.3680, -1.3823,
         1.7483,  1.2942, -0.7588, -0.7291, -1.2261, -0.8970,  0.8327,  1.1333,
         1.5878,  0.0891,  0.5280,  0.3989,  0.1955, -1.0455,  0.4364,  1.1388,
        -0.7819,  1.3072, -0.8736, -0.3792, -0.8247, -1.4188, -1.0088, -0.7348,
         1.9078, -1.1012, -0.7090,  0.3916, -0.3001,  0.3209,  0.2635,  0.8543,
         0.5985,  0.0758, -0.6523,  0.4882, -0.0912,  0.5008,  1.1756, -0.2584,
         0.3211,  0.3765, -1.2757,  0.8564,  0.0277, -0.1435,  0.3744, -1.2294,
        -0.7166, -0.2129, -0.9265,  0.0932, -1.9291,  1.2988,  0.0140, -0.0414,
         1.9615, -0.4395, -0.6023,  0.3582, -0.6587,  0.5204,  0.0948,  0.9241,
        -0.4053, -0.6732, -1.0680, -0.29